In [1]:
using Pkg
Pkg.activate("C:/Users/lisah/Documents/Repos/ma-code")

  Activating project at `C:\Users\lisah\Documents\Repos\ma-code`


In [ ]:
include("c:/Users/lisah/Documents/Repos/ma-code/src/src.jl")
using .Src

In [ ]:
#a_vals = [0.1, 0.9, 1.1, 1.7] 
#n0_vals = [0.4, 0.5, 1.0, 1.3, 2.3]

index_combos = [[1,1], [2,4], [3,5], [4,2], [4,3]]

path = "c:/Users/lisah/Documents/Repos/"
M_vals = [1000]
noise_vals = [0.0]
m = 0.45 # fix for now as given by literature
w0 = 0.95 # mean(a_vals)

gen_all_fish_data(M_vals, noise_vals, m, w0, path)

Dual{ForwardDiff.Tag{var"#13#14"{Hyperprm}, Float64}}(Dual{ForwardDiff.Tag{var"#13#14"{Hyperprm}, Float64}}(-8.611359883158884e-5,0.0005021272236991944,-0.0),Dual{ForwardDiff.Tag{var"#13#14"{Hyperprm}, Float64}}(0.0005021272236991944,-997.9934290233878,-0.0),Dual{ForwardDiff.Tag{var"#13#14"{Hyperprm}, Float64}}(-0.0,-0.0,-1.6847371963827007))[997.9934290233878 0.0; 0.0 1.6847371963827007]
Dual{ForwardDiff.Tag{var"#13#14"{Hyperprm}, Float64}}(Dual{ForwardDiff.Tag{var"#13#14"{Hyperprm}, Float64}}(-6.900837601676209e-6,7.837855753487844e-5,-0.0),Dual{ForwardDiff.Tag{var"#13#14"{Hyperprm}, Float64}}(7.837855753487844e-5,-997.9927815236128,-0.0),Dual{ForwardDiff.Tag{var"#13#14"{Hyperprm}, Float64}}(-0.0,-0.0,-1.6849763717162924))[997.9927815236128 0.0; 0.0 1.6849763717162924]
Dual{ForwardDiff.Tag{var"#13#14"{Hyperprm}, Float64}}(Dual{ForwardDiff.Tag{var"#13#14"{Hyperprm}, Float64}}(-2.6875091430213032e-6,-1.8814811734506187e-5,-0.0),Dual{ForwardDiff.Tag{var"#13#14"{Hyperprm}, Float64}}(-1.8

Excessive output truncated after 524382 bytes.

-8.530132873151356e-8,-0.0007710006517292571,5.345101894683025e-6),Dual{ForwardDiff.Tag{var"#13#14"{Hyperprm}, Float64}}(-0.0007710006517292571,-13720.623478041456,-12.672440967967981),Dual{ForwardDiff.Tag{var"#13#14"{Hyperprm}, Float64}}(5.345101894683025e-6,-12.672440967967981,-6.913715972184317))[13720.623478041456 12.672440967967981; 12.672440967967981 6.913715972184317]
Dual{ForwardDiff.Tag{var"#13#14"{Hyperprm}, Float64}}(Dual{ForwardDiff.Tag{var"#13#14"{Hyperprm}, Float64}}(-1.176798542592869e-7,-0.0044598824805102,-4.0133219200471505e-5),Dual{ForwardDiff.Tag{var"#13#14"{Hyperprm}, Float64}}(-0.0044598824805102,-9352.821787533858,-9.941649843656625),Dual{ForwardDiff.Tag{var"#13#14"{Hyperprm}, Float64}}(-4.0133219200471505e-5,-9.941649843656625,-6.693404221211277))[9352.821787533858 9.941649843656625; 9.941649843656625 6.693404221211277]
Dual{ForwardDiff.Tag{var"#13#14"{Hyperprm}, Float64}}(Dual{ForwardDiff.Tag{var"#13#14"{Hyperprm}, Float64}}(-2.3366294529699747e-7,-0.0060246189